In [ ]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [ ]:
tripsbird = pd.read_csv('../data/scooter_trips_bird.csv')

In [ ]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')
#saved as a geo dataframe (special geometry column)
print(zipcodes.crs)
zipcodes.head( )

In [ ]:
tripsbird.head()

In [ ]:
top_50_points = tripsbird.sort_values('tripdistance', ascending = False).head(50)

In [ ]:
top_50_points['startgeometry'] = top_50_points.apply(lambda x: Point((float(x.startlongitude), 
                                                         float(x.startlatitude))), 
                                        axis=1)
top_50_points.head(3)

In [ ]:
top_50_points['endgeometry'] = tripsbird.apply(lambda x: Point((float(x.endlongitude), 
                                                         float(x.endlatitude))), 
                                        axis=1)
top_50_points.head(3)

In [ ]:
top_start = top_50_points[['startlatitude', 'startlongitude']].copy()

In [ ]:
top_start['type'] = 'start'

In [ ]:
top_start = top_start.rename(columns = {'startlatitude': 'latitude', 'startlongitude': 'longitude'})

In [ ]:
top_end = top_50_points[['endlatitude', 'endlongitude']].copy()

In [ ]:
top_end['type'] = 'end'

In [ ]:
top_end = top_end.rename(columns = {'endlatitude': 'latitude', 'endlongitude': 'longitude'})

In [ ]:
top_end

In [ ]:
test = folium.Map(location = (top_latlong.loc[0,'latitude'], top_latlong.loc[0,'longitude']), zoom_start = 12)

for row_index, row_values in top_latlong.iterrows():
    if row_values['type'] == 'start':
        marker = folium.Marker(
            location = [row_values['geometry'].y, row_values['geometry'].x],
            popup = row_values['type'], icon = folium.Icon(color = 'green', icon = 'play', prefix = 'fa'))
        marker.add_to(test)
    if row_values['type'] == 'end':
        marker = folium.Marker(
            location = [row_values['geometry'].y, row_values['geometry'].x],
            popup = row_values['type'], icon = folium.Icon(color = 'red', icon = 'stop', prefix = 'fa'))
        marker.add_to(test)

test
        

In [ ]:
top_latlong = pd.concat([top_start, top_end])

In [ ]:
zipcodes.crs

In [ ]:
tripbird_geo = gpd.GeoDataFrame(top_latlong, 
                           crs = zipcodes.crs, 
                           geometry = top_latlong['geometry'])

In [ ]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [ ]:
startpoint_by_zip = gpd.sjoin(tripbird_geo, zipcodes, op = 'within')

In [ ]:
#fig, ax = matplotlib.subplots()
ax = zipcodes.plot(figsize = (8, 10), color = 'lightgreen')
startpoint_by_zip.plot( ax = ax, column = 'triprecordnum');
plt.show();

In [ ]:
center = zipcodes.geometry.centroid[27]
print(center)
type(center)

In [ ]:
area_center = list([center.y, center.x])

print(area_center)

In [ ]:
map_nash = folium.Map(location = area_center, zoom_start = 12)

In [ ]:
i = 0
for row_index, row_values in startpoint_by_zip.iterrows():
    if i <3: 
        print('index is', row_index)
        print('values are:')
        print(' ')
        print(row_values)
        print('------------------------- ')
        i+=1

In [ ]:
map_nash = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(zipcodes).add_to(map_nash)

#iterate through stops_in_37207 to create locations and markers 
#for each bus stop
#remember for Folium locations, lat is listed first!!

for row_index, row_values in top_latlong.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    pop = str(row_values['type'])
    icon=folium.Icon(color="green",icon="scooter", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop, icon = icon) 

    marker.add_to(map_nash)
map_nash

#map_nash.save('../data/map_nash.html')

In [ ]:
#create a marker cluster
marker_cluster = MarkerCluster().add_to(map_nash)

folium.GeoJson(zipcodes).add_to(map_nash)

# inside the loop add each marker to the cluster
for row_index, row_values in startpoint_by_zip.iterrows():
    loc = [row_values['startlatitude'], row_values['startlongitude']]
    pop = str(row_values['triprecordnum'])
    icon=folium.Icon(color="blue",icon="scooter", prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
    icon = icon) 
    
    marker.add_to(marker_cluster)
marker_cluster
marker_cluster.save('../data/marker_cluster.html')